In [84]:
# Imports
from credentials import password, email
from data import stock_list_one
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import datetime, re, requests, io, time, random, string

In [85]:
driver = webdriver.Chrome('/Users/Simone/Coding/chromedriver')
time.sleep(3)

In [86]:
def sign_in(email=email, password=password):
   driver.get('https://wallmine.com')
   time.sleep(2)
   driver.find_element_by_xpath('/html/body/main/header/div/ul/li[1]/ul/li[3]/a').click() # click sign in
   time.sleep(1)
   driver.find_element_by_xpath('//*[@id="new_user"]/div[5]/div[1]/div[2]/a').click() #sign in with password
   # log in
   # verify on sign in page: 
   if "We're glad you're back!" in driver.page_source: 
       driver.find_element_by_xpath('//*[@id="user_email"]').send_keys(email)
       driver.find_element_by_xpath('//*[@id="user_password"]').send_keys(password)
       time.sleep(0.2)
       driver.find_element_by_xpath('//*[@id="new_user"]/div[5]/div[2]/div[1]/button').click()
       time.sleep(3)
       if 'US Markets' in driver.page_source: print('successfully signed in')
   else: 
        print('wrong page, start over')
        sign_in()

In [87]:
sign_in(email, password)

successfully signed in


In [96]:
data_to_store = {
    "company_name" : '/html/body/main/section/div[2]/div/div[1]/h1/div[2]/a',
    'current_price' : '/html/body/main/section/div[3]/div/div/div/div/div[2]/div/div[1]/span[1]',
    'percentage' : '/html/body/main/section/div[3]/div/div/div/div/div[2]/div/div[2]/div',
    'amount_changed' : '/html/body/main/section/div[3]/div/div/div/div/div[2]/div/div[1]/span[2]',
    'market_cap' : '/html/body/main/section/div[4]/div[1]/div[2]/div[1]/div[1]/table/tbody/tr[1]/td',
    'enterprise_value' : ' /html/body/main/section/div[4]/div[1]/div[2]/div[1]/div[1]/table/tbody/tr[2]/td',
    'income' : '/html/body/main/section/div[4]/div[1]/div[2]/div[1]/div[2]/table/tbody/tr[3]/td', 
    'revenue' : '/html/body/main/section/div[4]/div[1]/div[2]/div[1]/div[2]/table/tbody/tr[1]/td', 
    'ebitda' : '/html/body/main/section/div[4]/div[1]/div[2]/div[1]/div[2]/table/tbody/tr[2]/td',
    'volume' : '/html/body/main/section/div[4]/div[1]/div[2]/div[2]/div[1]/table/tbody/tr[1]/td',
    'relative_volume' : '/html/body/main/section/div[4]/div[1]/div[2]/div[2]/div[1]/table/tbody/tr[2]/td'
    }


In [91]:
def get_data(stock_array, criteria):
    # stock array is array of dicts with keys 'exchange' and 'stock_ticker'
    # criteria is dict of data you with to store with values of xpaths
    for stock in stock_array:
        driver.get(f"https://wallmine.com/{stock.get('exchange')}/{stock.get('stock_ticker')}")
        time.sleep(2)
        company_data = {}
        for attribute in criteria:
            company_data[attribute] = driver.find_element_by_xpath(criteria[attribute]).text
            # get percentage up or down
            if attribute == 'percentage': 
                if driver.find_element_by_xpath(criteria['percentage']).get_attribute('class') == 'badge badge-danger': company_data['percentage'] = '-' + company_data['percentage']
                else: company_data['percentage'] = '+' + company_data['percentage']
            if attribute in ['market_cap', 'income', 'revenue', 'ebitda', 'enterprise_value']:
                if company_data[attribute][-1] == 'B': 
                    company_data[attribute] = float(company_data[attribute][1:-1]) * 1000000000
                elif company_data[attribute][-1] == 'M': 
                    company_data[attribute] = float(company_data[attribute][1:-1]) * 1000000
                elif company_data[attribute][-1] == 'T':
                    company_data[attribute] = float(company_data[attribute][1:-1]) * 1000000000000
        print(company_data)


In [99]:
get_data(stock_list_one[:5], data_to_store)

{'company_name': 'Apple Inc', 'current_price': '$127.83', 'percentage': '+0.54%', 'amount_changed': '+0.69', 'market_cap': 1970000000000.0, 'enterprise_value': 2200000000000.0, 'income': 57411000000.0, 'revenue': 274515000000.0, 'ebitda': 77344000000.0, 'volume': '85.222M / 99.692M', 'relative_volume': '0.85 ×'}
{'company_name': 'Microsoft Corporation', 'current_price': '$216.44', 'percentage': '+1.78%', 'amount_changed': '+3.79', 'market_cap': 1610000000000.0, 'enterprise_value': 1610000000000.0, 'income': 47496000000.0, 'revenue': 147114000000.0, 'ebitda': 55823000000.0, 'volume': '25.207M / 24.952M', 'relative_volume': '1.01 ×'}
{'company_name': 'Amazon.com Inc.', 'current_price': '$3,135.00', 'percentage': '+0.99%', 'amount_changed': '+30.75', 'market_cap': 1510000000000.0, 'enterprise_value': 1590000000000.0, 'income': 17377000000.0, 'revenue': 347945000000.0, 'ebitda': 31875000000.0, 'volume': '3.017M / 3.671M', 'relative_volume': '0.82 ×'}
{'company_name': 'Alphabet Inc', 'curre

In [ ]:
from pymongo import MongoClient

client = MongoClient()
db = client['stock_data']
collection = db['current_data']

db.current_data.insert(stock_object)
db.current_data.findOne()